In [1]:
# NSE scanner and pickler

# STATUS: Completed
# Run-time: 5 hours

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=1) # kavi tws live
# ib = IB().connect('127.0.0.1', 3000, clientId=10) # kavi IBG live

In [2]:
import pandas as pd

#... assignments
exchange = 'NSE'

#... build underlying contracts
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_paisa
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_paisa.ibSymbol if s not in indexes+discards])

symbols = equities+indexes

c = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]
contracts = ib.qualifyContracts(*c)

In [13]:
contracts = contracts[-4:] # DATA LIMITER!!!

In [ ]:
#... get ohlc
# extract ohlc with symbol
ohlc = {c.symbol: ib.reqHistoricalData(contract= c, endDateTime='', durationStr='12 M', barSizeSetting='1 day', 
                                       whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True) for c in contracts}

In [112]:
# create a dataframe
df_ohlc = pd.concat([util.df(v).assign(symbol=k) for k, v in ohlc.items()], axis = 0)
df_ohlc.date = pd.to_datetime(df_ohlc.date)  # convert to datettimeindex

In [128]:
df_ohlc.rolling(3).std()

NotImplementedError: ops for Rolling for this dtype datetime64[ns] are not implemented

In [111]:
df_ohlc.groupby('symbol')['high'].resample('W', level=0).

Exception: Column(s) high already selected

In [96]:
df_ohlc.groupby('symbol').high

In [95]:
import numpy as np
np.random.seed(123)
df1 = pd.DataFrame({ 'price':abs(np.random.randn(5)).cumsum(), 'ticker':'GOOG_L' }, 
                   index=pd.date_range('1-4-2017', periods=5, freq='B') )
df2 = pd.DataFrame({ 'price':abs(np.random.randn(5)).cumsum(), 'ticker':'AAPL' }, 
                   index=pd.date_range('1-4-2017', periods=5, freq='B') )
df = df1.append(df2).sort_index()
df

,price,ticker
2017-01-04,1.085631,GOOG_L
2017-01-04,1.651437,AAPL
2017-01-05,2.082976,GOOG_L
2017-01-05,4.078116,AAPL
2017-01-06,2.365955,GOOG_L
2017-01-06,4.507028,AAPL
2017-01-09,3.872249,GOOG_L
2017-01-09,5.772965,AAPL
2017-01-10,4.450850,GOOG_L
2017-01-10,6.639705,AAPL


In [ ]:
#... Get risk-free rate from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
rate = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100